In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import seaborn as sns

In [2]:
# Read in data: features & labels
df_features_dict = {}
df_labels_dict = {}

num_feature_files = 15
num_label_files = 15

for i in range(1, num_feature_files):
    df_features_dict[i] = pd.read_csv(f'lcof_hourly_features_{i}.csv')

for i in range(1, num_label_files):
    df_labels_dict[i] = pd.read_csv(f'lcof_hourly_labels_{i}.csv')

In [3]:
df_features_frames = [df_features_dict[1], df_features_dict[2], df_features_dict[3],
                      df_features_dict[4], df_features_dict[5], df_features_dict[6],
                      df_features_dict[7], df_features_dict[8], df_features_dict[9],
                      df_features_dict[10], df_features_dict[11], df_features_dict[12],
                      df_features_dict[13], df_features_dict[14]
                     ]

df_labels_frames = [df_labels_dict[1], df_labels_dict[2], df_labels_dict[3],
                    df_labels_dict[4], df_labels_dict[5], df_labels_dict[6],
                    df_labels_dict[7], df_labels_dict[8], df_labels_dict[9],
                    df_labels_dict[10], df_labels_dict[11], df_labels_dict[12],
                    df_labels_dict[13], df_labels_dict[14]
                   ]

df_features = pd.concat(df_features_frames, ignore_index=True)
df_labels = pd.concat(df_labels_frames, ignore_index=True)

df_features.drop('num_plans', axis=1, inplace=True)
df_labels.drop('num_plans', axis=1, inplace=True)

df = pd.merge(df_features, df_labels,
              how='inner',
              left_on=['datestr', 'city_id', 'hour_of_day'],
              right_on=['datestr', 'city_id', 'hour_of_day']
             )

In [4]:
# Filter out outliers
df_clean = df[df['hour_gb_gamma_95'] < 3].reset_index(drop=True)
print(f"The original dataset size = {len(df)}; the new dataset size = {len(df_clean)}")

The original dataset size = 20933; the new dataset size = 20509


In [5]:
# Labels
gb_100 = df_clean['hour_gb_gamma_100']
gb_99 = df_clean['hour_gb_gamma_99']
gb_95 = df_clean['hour_gb_gamma_95']
gb_90 = df_clean['hour_gb_gamma_90']

fare_100 = df_clean['hour_fare_gamma_100']
fare_99 = df_clean['hour_fare_gamma_99']
fare_95 = df_clean['hour_fare_gamma_95']
fare_90 = df_clean['hour_fare_gamma_90']

y = gb_95

In [6]:
# Features
X = df_clean.loc[:, ['market_log_cr',
                     'market_log_eta',
                     'market_log_fare_p50_scale',
                     'market_surge'
                    ]
                ]

In [7]:
# Training & Test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=116)

In [8]:
def calculate_loss(x1, x2, x3, x, y, b0, b1, b2, b3):
    alpha = 1 - 1/np.maximum(x, 1)**b3
    return np.mean((b0 + b1*x1 + b2*x2 + alpha*x3 - y)**2 / 2)

In [9]:
def grad(x1, x2, x3, x, y, b0, b1, b2, b3):
    alpha = 1 - 1/np.maximum(x, 1)**b3
    re = b0 + b1*x1 + b2*x2 + alpha*x3 - y
    
    db0 = np.mean(re)
    db1 = np.mean(re * x1)
    db2 = np.mean(re * x2)
    db3 = np.mean(re * x3 * b3 * np.maximum(x, 1)**(-b3-1))
    
    return db0, db1, db2, db3

In [10]:
def main(n_iter=20, step_size=400, lr=0.01, tol=1e-03):
    N = len(X_train)
    num_iters = N // step_size
    
    b0 = 0.0
    b1 = 1.0
    b2 = 2.0
    b3 = 1.0
    
    train_losses = []
    test_losses = []
    
    best_b0 = b0
    best_b1 = b1
    best_b2 = b2
    best_b3 = b3
    
    best_test_loss = np.inf
    best_iter = 0
    
    for it in range(n_iter):
        print(f"We are in iteration: {it+1} ...")
        
        # Reshuffle
        ids = list(range(N))
        np.random.shuffle(ids)
        
        X_train_ids = X_train.reset_index(drop=True)
        y_train_ids = y_train.reset_index(drop=True)
        
        train_loss = 0
        for idx in range(num_iters):
            y = y_train_ids[ids][idx*step_size:(idx+1)*step_size]
            x1 = X_train_ids['market_log_cr'][ids][idx*step_size:(idx+1)*step_size]
            x2 = X_train_ids['market_log_eta'][ids][idx*step_size:(idx+1)*step_size]
            x3 = X_train_ids['market_log_fare_p50_scale'][ids][idx*step_size:(idx+1)*step_size]
            x = X_train_ids['market_surge'][ids][idx*step_size:(idx+1)*step_size]
            
            cur_loss = calculate_loss(x1, x2, x3, x, y, b0, b1, b2, b3)
            train_loss += cur_loss
            
            db0, db1, db2, db3 = grad(x1, x2, x3, x, y, b0, b1, b2, b3)
            
            if np.all(np.abs([lr*db0, lr*db1, lr*db2, lr*db3]) <= tol):
                break
            
            b0 -= lr * db0
            b1 -= lr * db1
            b2 -= lr * db2
            b3 -= lr * db3
        
        train_loss /= num_iters
        print(f"Train Loss = {train_loss}")
        
        train_losses.append(train_loss)
        
        # Test loss
        y = y_test
        x1 = X_test['market_log_cr']
        x2 = X_test['market_log_eta']
        x3 = X_test['market_log_fare_p50_scale']
        x = X_test['market_surge']
        test_loss = calculate_loss(x1, x2, x3, x, y, b0, b1, b2, b3)
        
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            best_iter = it

            best_b0 = b0
            best_b1 = b1
            best_b2 = b2
            best_b3 = b3
        
        print(f"Test Loss = {test_loss}")
        
        test_losses.append(test_loss)
    
    return best_b0, best_b1, best_b2, best_b3, best_test_loss, best_iter

In [13]:
best_b0, best_b1, best_b2, best_b3, best_test_loss, best_iter = main(n_iter=100, lr=0.01, step_size=400)

We are in iteration: 1 ...
Train Loss = 2.240583912689002
Test Loss = 1.2694832042738755
We are in iteration: 2 ...
Train Loss = 0.8524069818960224
Test Loss = 0.5667851869333339
We are in iteration: 3 ...
Train Loss = 0.44763645496636906
Test Loss = 0.3538453644433986
We are in iteration: 4 ...
Train Loss = 0.3217792809653806
Test Loss = 0.2812166598652775
We are in iteration: 5 ...
Train Loss = 0.2743957681108104
Test Loss = 0.2488689777848725
We are in iteration: 6 ...
Train Loss = 0.24959195533458386
Test Loss = 0.22906880278937847
We are in iteration: 7 ...
Train Loss = 0.23198174672013303
Test Loss = 0.21368926386867979
We are in iteration: 8 ...
Train Loss = 0.04540479300887465
Test Loss = 0.21112446541782745
We are in iteration: 9 ...
Train Loss = 0.10508775388092803
Test Loss = 0.2040166980367939
We are in iteration: 10 ...
Train Loss = 0.12904449239825538
Test Loss = 0.19652619256256382
We are in iteration: 11 ...
Train Loss = 0.025375777930928087
Test Loss = 0.19526408357386

In [14]:
print(best_b0, best_b1, best_b2, best_b3, best_test_loss, best_iter)
# gamma = 0.95, fare_scaling = p50
# -- 1.386476124588059 0.05494968988410009 0.4444485446726071 1.3957356824560634 0.1446741220723692 49
# -- 1.3664190120791024 0.046007235456259656 0.4023823860464308 1.305831223967143 0.13642031467885057 99

1.3664190120791024 0.046007235456259656 0.4023823860464308 1.305831223967143 0.13642031467885057 99
